In [14]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import json

In [2]:
dataset_peoples = pd.read_csv('../data/peoples.csv')
dataset_pets = pd.read_csv('../data/dataset_union.csv')

In [3]:
uuids_peoples = dataset_peoples['ID'].to_list()
ids_pets = dataset_pets['ID'].to_list()

In [4]:
from collections import namedtuple

Pet = namedtuple('Pet', ['id', 'animal_name', 'animal_gender', 'breed_name', 'borough', 'zip_code', 'type', 'image'])

In [5]:
print(len(dataset_pets))

134291


In [6]:
dict_people_pet = {}

# garantindo que todo usuário terá um animal
for index, id in tqdm(enumerate(uuids_peoples), total=len(uuids_peoples)):
    pet = dataset_pets.loc[dataset_pets.index[index]]
    dict_people_pet[f"{id}"] = [Pet(id = pet['ID'], animal_name = pet['AnimalName'], animal_gender = pet['AnimalGender'], breed_name = pet['BreedName'], borough = pet['Borough'],  zip_code = str(pet['ZipCode']), type = pet['Type'], image = pet['Image'])]
    # if index in dataset_pets.index:
    #     dataset_pets = dataset_pets.drop(index)

100%|██████████| 24500/24500 [00:02<00:00, 8273.40it/s]


In [7]:
count_pets = len(ids_pets)
count_users = len(uuids_peoples)
people_keys = list(uuids_peoples)

with tqdm(total=count_pets, desc="Handing out pets...") as inner_tqdm:
    for user_id in people_keys:
        quantity_pets = np.random.randint(1, 9)
        
        pet_indices = np.random.choice(dataset_pets.index, size=quantity_pets, replace=False)
        pets = []
        
        for pet_index in pet_indices:
            pet = dataset_pets.loc[pet_index]
            pets.append(Pet(id=pet['ID'], animal_name=pet['AnimalName'], animal_gender=pet['AnimalGender'], breed_name=pet['BreedName'], borough=pet['Borough'], zip_code=str(pet['ZipCode']), type=pet['Type'], image=pet['Image']))

        dict_people_pet[user_id].extend(pets)
        inner_tqdm.update(quantity_pets)

        if len(dataset_pets) < quantity_pets:
            break

            # Marca os animais que foram distribuídos como indisponíveis
        dataset_pets = dataset_pets.drop(index=pet_indices)

print("Done handing out pets!")
        

Handing out pets...:  82%|████████▏ | 110193/134291 [06:09<01:20, 298.15it/s]

Done handing out pets!


In [11]:
len(dict_people_pet)

24500

In [12]:
count_pets = 0

for uuid_user in dict_people_pet:
    for pet in dict_people_pet[uuid_user]:
        count_pets += 1

print(count_pets)

134693


In [ ]:
dict_converted = {}
for key, value in dict_people_pet.items():
    dict_converted[key] = [nt._asdict() for nt in value]

In [27]:
print(len(dict_converted))

24500


In [31]:
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NumpyEncoder, self).default(obj)

with open('../data/people_and_pet.json', 'w') as f:
    json.dump(dict_converted, f, cls=NumpyEncoder)